<a href="https://colab.research.google.com/github/relew/LLMZoomCamp/blob/main/03_dlt_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro dlt -> LanceDB loading example

https://lu.ma/cnpdoc5n

If you want to play around with this notebook and make edits in future, we highly recommend making a copy since the link is view only! Also make sure you're signed in with your Google account to be able to add secrets.

Before going into a more complex example, we will go through a simple example of how to load the course Q&A data into LanceDB.

## Install requirements

To create a json -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [1]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

## Load the data

We'll first load the data just into LanceDB, without embedding it. LanceDB stores both the data and the embeddings, and can also embed data and queries on the fly.

Some definitions:
* A dlt **source** is a grouping of **resources** (e.g. all your data from Hubspot)
* A dlt **resource** is a function that yields data (e.g. a function yielding all your Hubspot companies)
* A dlt **pipeline** is how you ingest your data

Loading the data consists of a few steps:
1. Use the requests library to get the data
2. Define a dlt resource that yields the individual documents
3. Create a dlt pipeline and run it

In [2]:
import requests
import dlt

qa_dataset = requests.get("https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1").json()

@dlt.resource
def qa_documents():
  for course in qa_dataset:
    yield course["documents"]

pipeline = dlt.pipeline(pipeline_name="from_json", destination="lancedb", dataset_name="qanda")

load_info = pipeline.run(qa_documents, table_name="documents")

print(load_info)

_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': False}, {'name': 'schema', 'data_type': 'text', 'nullable': False}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type':

In [3]:
import lancedb

db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents']


In [4]:
db_table = db.open_table("qanda___documents")

db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,576318df-078a-51ee-ae77-4bc90716e930,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721631838.3735762,84fgKz+utY7rFg
1,a5dc1681-17b3-5679-94b2-994a6e9a5321,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721631838.3735762,RFrxQQQP2GzV7w
2,47594130-3282-529d-aba8-07bd011d4f9f,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721631838.3735762,LtXHjNmp+Hk65A
3,00651038-4eb4-562d-a9db-b9ecc03bbf54,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721631838.3735762,HDHHhcC3+uWkuA
4,1b9bd092-f144-53b1-93a4-d1ce539f7428,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721631838.3735762,HOUCnsUP4se9rA
...,...,...,...,...,...,...
943,6b251e1e-d6b2-56cf-aaa0-b3da1e7d1609,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721631838.3735762,G2Dwrb9/d2ts8A
944,d8b33cb4-7008-5c10-b66d-30d5ba99ce44,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721631838.3735762,4Jk1nUblZVLuxA
945,bb912476-32c3-5e32-b75e-f347727e8e62,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721631838.3735762,EN4V14pQQDHM4A
946,de7205cb-a02a-5b1e-a348-c55030a5f272,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721631838.3735762,X7kvXx07t3eh/A


## Load and embed the data

Now we load the same data again (into a new table), but embed it directly with the `lancedb_adapter`. This consists of the following steps:

1. Define the embedding model to use via ENV variables
2. Define a new pipeline to load the same data and embed the "text" and "question" columns with the `lancedb_adapter`

You can use any embedding model, from open source to OpenAI. We've chosen the [`all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) sentence transformer for speed and simplicty.

Note: this pipeline runs slightly longer because it has to download the model and embed the data.

In [5]:
import os
from dlt.destinations.adapters import lancedb_adapter

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

pipeline = dlt.pipeline(pipeline_name="from_json_embedded", destination="lancedb", dataset_name="qanda_embedded")

load_info = pipeline.run(lancedb_adapter(qa_documents, embed=["text", "question"]), table_name="documents")
print(load_info)

_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': False}, {'name': 'schema', 'data_type': 'text', 'nullable': False}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type':

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

UPLOAD
Pipeline from_json_embedded load step completed in 28.91 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda_embedded
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7ff8fdeb53f0> location to store data
Load package 1721631840.301809 is LOADED and contains no failed jobs


In [6]:
db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [7]:
db_table = db.open_table("qanda_embedded___documents")

db_table.to_pandas()

,id__,vector__,text,section,question,_dlt_load_id,_dlt_id
0,2042447c-32c6-5a57-a950-c763438b2438,"[-0.00035095983, -0.062014256, -0.03799991, 0....",The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721631840.301809,kyPCWzHeExenJg
1,3f35cb6f-35ed-5b3f-bf11-2f84ce0c016f,"[0.020011423, -0.011535534, 0.0130172055, -0.0...",GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721631840.301809,QDA8rPUz0Ty6uw
2,77a45b5a-4022-5d4e-b66d-bf7d1c9cdb68,"[0.014857549, -0.06664994, -0.013571203, 0.023...","Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721631840.301809,T0t6N7zpA5LgtQ
3,656f9261-50f7-58c9-ac80-8739585abb0a,"[-0.023312101, -0.09461491, 0.056361604, -0.00...",You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721631840.301809,5GzN+1xganmCFA
4,2e888539-d650-5371-a4f0-8cd41f336821,"[0.026537659, -0.017796658, 0.0021156375, 0.00...",You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721631840.301809,+dxdWfyZtLaIRQ
...,...,...,...,...,...,...,...
943,313ffa3b-2b00-51fe-bdff-d26fd563f53c,"[0.016619304, -0.033603117, -0.09334721, -0.02...",Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721631840.301809,1DcSgc1GNqhmOw
944,60d547e9-c196-5acb-bbef-83e008552d7d,"[0.026872871, -0.0019949335, 0.008369085, -0.0...",Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721631840.301809,bWStOWB3PJsTVA
945,06950ca8-87e3-566c-a0f8-d0680e75dd58,"[0.03513752, 0.056265578, 0.024428517, -0.0651...",Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721631840.301809,dMqKNNoFT754IQ
946,a5b3d6b0-6507-5ebc-95a1-4c928dc31b62,"[0.03380982, -0.0031219546, 0.0017484119, 0.01...",Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721631840.301809,Arw87r8NiFk5Rg


That's all for this intro example! The DB could now be used as a basis for a RAG.

# Create an up-to-date RAG with dlt and LanceDB

In this demo, we will be creating an LLM chat bot that has the latest knowledge of the employee handbook of a fictional company. We will be able to chat to it about specific policies like PTO, work from home etc.

To build this, we would need to do three things:
1. The company policies exist in a [Notion Page](https://dlthub.notion.site/Employee-handbook-669c2a1e04044465811c8ca22977685d). We will need to first extract the text from these pages.
2. Once extracted, we will want to embed them into vectors and then store them in a vector database.
3. This will allow us to create our RAG: a function that would accept a user question, match it to the information stored in the vector database, and then send the question + relevant information as input to the LLM.

We will be using the following OSS tools for this:
1. dlt for data ingestion:  
  1. dlt can easily connect to any REST API source (like Notion)
  2. It also has integrations with vector databases, like LanceDB.
  3. It also allows to easily plug in functionality like incremental loading.
2. LanceDB as a vector database:
  1. LanceDB is an open-source vector database that is very easy to use and integrate into python workflows
  2. It is in-process and serverless (like DuckDB), which makes querying and retreival very efficient
3. Ollama for RAG:
  1. Ollama is open-source and allows you to easily run LLMs locally

**Note on running this notebook**: We are going to download and use a local Ollama instance for the RAG, so preferably select the **T4 GPU** in the runtime when starting this notebook (Runtime > Change runtime type > Hardware accelerator > T4 GPU).

You can also use the default CPU in case you're facing technical issues, but then your LLM responses might be slower (~2 mins/response)

## Part 1: Create a Notion -> LanceDB pipeline using dlt

### 1. Install requirements

To create a notion -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [8]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

### 2. Create a dlt project with rest_api source and lancedb destination

We now create a dlt project using the command `dlt init <source> <destination>`.

This downloads all the modules required for the dlt source (rest api, in this case) into the local directory. See the side panel for the directory structure created.

What is the dlt rest api source?

It is a dlt source that allows you to connect to any REST API endpoint using a declarative configuration. You can:
- pass the endpoints that you want to connect to,
- define the relation between the endpoints
- define how you want to handle pagination and authentication

In [9]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


### 3. Add API credentials

To access APIs, databases, or any third-party applications, one might need to specify relevant credentials.

With dlt, we can do it in two ways:
1. Pass the credentials and any other sensitive information inside `.dlt/secrets.toml`
  ```toml
  [sources.rest_api.notion]
  api_key = "notion api key"

  [destination.lancedb]
  embedding_model_provider = "sentence-transformers"
  embedding_model = "all-MiniLM-L6-v2"

  [destination.lancedb.credentials]
  uri = ".lancedb"
  api_key = "api_key"
  embedding_model_provider_api_key = "embedding_model_provider_api_key"
  ```
2. Pass them as environment variables
  ```python
  import os
  
  os.environ["SOURCES__REST_API__NOTION__API_KEY"] = "notion api key"

  os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
  os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

  os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"
  os.environ["DESTINATION__LANCEDB__CREDENTIALS__API_KEY"] = "api_key"
  os.environ["DESTINATION__LANCEDB__CREDENTIALS__EMBEDDING_MODEL_PROVIDER_API_KEY"] = "embedding_model_provider_api_key"
  ```

In [17]:
from google.colab import userdata
userdata.get('secretName')

SecretNotFoundError: Secret secretName does not exist.

We are going to be using option 2. It's not advisable to paste sensitive information like API keys inside the code, so instead we're going to include them inside the secrets tab in the side panel of the notebook. This will allow us to access the secret values from the notebook.

Since we are using the OSS version of LanceDB and OSS embedding models, we only need to specify the API key for Notion.

**Note**: You will need to copy the [notion API key](https://share.1password.com/s#da9KgMwPaZUaey3WCaD7ICJoyHDGd3Xos2EZ29WrSWQ) into the secrets tab under the name `SOURCES__REST_API__NOTION__API_KEY`. Make sure to enable notebook access after pasting the key.

In [18]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

### 4. Write the pipeline code

**Note**: We first go over the code step by step before putting it into runnable cells

1. Import necessary modules (run this cell)

In [19]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

2. Configure the dlt rest api source to connect to and extract the relevant data out from the Notion REST API.

  Our notion space has multiple pages and each page has multiple paragraphs (called blocks). To extract all this data from the Notion API, we would first need to get a list of all the page_ids (each page has a unique page_id), and then use the page_id to request the contents from the individual pages. Specifically:
  1. We will first request the page_ids from the `/search` endpoint
  2. And then using the returned page_ids, we will request the contents from the `/blocks/{page_id}/children` endpoint

  With this in mind, we can configure the dlt notion rest api source as follows:
  ```python
  RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }
    ```
    Explanation:
    1. `client`: Here we added our base url, headers, and authentication
    2. `resources`: This is a list of endpoints that we wish to request data from (here: `/search` and `/blocks/{page_id}/children`)
    3. [`/search`](https://developers.notion.com/reference/post-search) endpoint:
      - The Notion API search endpoint allows us to filter pages based on the title. We can specify which pages we want returned based on the parameter "query". For example, if we'd like to return only those pages which has the word "workshop" in the title, then we would set `"query": "workshop"` in the json body.    
      - As a response, it returns only page metadata (like page_id). Example response:
      ```json
          {
            "object": "list",
            "results": [
              {
                "object": "page",
                "id": "954b67f9-3f87-41db-8874-23b92bbd31ee",
                "created_time": "2022-07-06T19:30:00.000Z",
                "last_edited_time": "2022-07-06T19:30:00.000Z",
                .
                .
                .
            ],
            "next_cursor": null,
            "has_more": false,
            "type": "page_or_database",
            "page_or_database": {}
          }
      ```
      - This is how we would define our endpoint configuration for `/search`:
      ```python
           {
             "name": "search",
             "endpoint": {
                 "path": "search",
                 "method": "POST",
                 "paginator": PostBodyPaginator(),
                 "json": {
                     "query": "workshop",
                     "sort": {
                         "direction": "ascending",
                         "timestamp": "last_edited_time"
                     }
                 },
                 "data_selector": "results"
             }
         },
      ```
      - `paginator` allows us to specify the pagination strategy relevant for the API and the endpoint. (More on this later)
      - Since `/search` is a POST endpoint, we can include the json body inside the key `json`.
      - We don't need the whole JSON response, but only the contents inside the field "results". We filter this out by specifying `"data_selector": "results"`.
    4. [`blocks/{page_id}/children`](https://developers.notion.com/reference/get-block-children) endpoint:
      - This is a GET point that returns a list of block objects (in our case, paragraphs) from a specific page.
      - Since it accepts page_id as a parameter, we can pass this inside the key `params`
      - We would like to be able to automatically fetch the page_ids returned from the `/search` endpoint and pass it as parameter into the endpoint `blocks/{page_id}/children`. We can do this by linking the two resources as follows:
      ```python
      {
            "name": "page_content",
            "endpoint": {
                "path": "blocks/{page_id}/children",
                "paginator": JSONResponsePaginator(),
                "params": {
                    "page_id": {
                        "type": "resolve",
                        "resource": "search",
                        "field": "id"
                    }
                },
            }
      }
      ```
      - By specifying `"type":"resolve"`, we are letting dlt know that this parameter needs to be resolved from the parent resource `"search"` using the field `"id"`, which corresponds to the page id in the response of `/search`.

  Note on pagination:

  Different REST APIs might use different strategies to handle paginated responses. dlt has built-in support for [most common pagination mechanisms](https://dlthub.com/docs/general-usage/http/rest-client#paginators), and these can be explicity passed inside the configuration like shown above.

  However in most cases, it won't be necessary to explicity specify the pagination strategy, since dlt detects it automatically.

  In case the specific pagination is not supported by dlt yet, then you can also implement a custom paginator. For example, dlt does not have a built-in paginator for POST methods, so we write our own paginator. We take the [code provided in the docs for it](https://dlthub.com/docs/general-usage/http/rest-client#example-2-creating-a-paginator-for-post-requests), and make small modifications to it based on the [notion API documentation](https://developers.notion.com/reference/intro#parameters-for-paginated-requests).

  ```python
  class PostBodyPaginator(BasePaginator):
      def __init__(self):
          super().__init__()
          self.cursor = None

      def update_state(self, response: Response) -> None:
          # Assuming the API returns an empty list when no more data is available
          if not response.json():
              self._has_next_page = False
          else:
              self.cursor = response.json().get("next_cursor")
              if self.cursor is None:
                  self._has_next_page = False

      def update_request(self, request: Request) -> None:
          if request.json is None:
              request.json = {}

          # Add the cursor to the request body
          request.json["start_cursor"] = self.cursor
  ```

3. Extract relevant content from the response body

  The response returned from the API is a nested JSON which we need to pre-process before using it anywhere. dlt can unnest json automatically, but since the Notion API is a little tricky it's better to pre-process this first so we have a more structured DB as a result.
  
  One way to do this is to pass the JSON response through a transformation function that will extract only the relevant data from the JSON body (we later add this as a mapping to the resource):

  ```python
  def extract_page_content(response):
      block_id = response["id"]
      last_edited_time = response["last_edited_time"]
      block_type = response.get("type", "Not paragraph")
      if block_type != "paragraph":
          content = ""
      else:
          try:
              content = response["paragraph"]["rich_text"][0]["plain_text"]
          except IndexError:
              content = ""
      return {
          "block_id": block_id,
          "block_type": block_type,
          "content": content,
          "last_edited_time": last_edited_time,
          "inserted_at_time": datetime.now(timezone.utc)
      }
  ```
  This is also where you could implement some sort of chunking strategy, but we will omit this in this example as the Notion text is already pre-chunked into paragraphs. Any data pre-processing can also happen here.

  **Note**: If you want to include the parent page in the returned data, you can do so by including `response["parent"]["page_id"]`. See 200 response example in the [Notion docs](https://developers.notion.com/reference/get-block-children).
  
  JSON response before the function:
  ```
  {
      "object": "list",
      "results": [
        {
          "object": "block",
          "id": "c02fc1d3-db8b-45c5-a222-27595b15aea7",
          "created_time": "2022-03-01T19:05:00.000Z",
          "last_edited_time": "2022-03-01T19:05:00.000Z",
          .
          .
          .
          "type": "paragraph",
          "paragraph": {
            "rich_text": [
              {
                .
                .
                .
                "annotations": {
                  .
                  .
                  .

                },
                "plain_text": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
                "href": "https://en.wikipedia.org/wiki/Lacinato_kale"
              }
            ],
            "color": "default"
          }
        }
      ],
      "next_cursor": null,
      "has_more": false,
      "type": "block",
      "block": {}
  }
  ```
  After passing it through the transformation function:

  ```
  {
      "block_id": "c02fc1d3-db8b-45c5-a222-27595b15aea7",
      "block_type": "paragraph",
      "content": "Lacinato kale is a variety of kale with a long tradition in Italian cuisine, especially that of Tuscany. It is also known as Tuscan kale, Italian kale, dinosaur kale, kale, flat back kale, palm tree kale, or black Tuscan palm.",
      "last_edited_time": "2022-03-01T19:05:00.000Z",
  }

    ```

4. Load the data incrementally

  Incremental loading is a very important aspect of building scalable data pipelines. It is the technique of loading only the new or changed data since the last run of the pipeline.

  In our case, when we first run the pipeline, all paragraphs from the employee handbook will get loaded as separate rows inside a lancedb table. Now if we change the content of one of the paragraphs and re-run the pipeline to update the table, then without doing incremental loading, one of two things may happen depending on the option we choose:
  - If we choose the option "replace", then the existing data in lancedb will be dropped, and all of the paragraphs will be reloaded.
  - If we choose the option "append", then the existing rows would remain and all of the paragraphs would be loaded again as new rows resulting in twice as many rows

  To ensure that only the new/changed rows are loaded we would need the following pieces:
  - A column that can keep track of changes in the row (Example: only load rows where `last_edited_time` is greater than the current maximum `last_edited_time`)
  - A primary_key column that uniquely identify a row, so it's possible to track when the row changes
  - A strategy to resolve changes in a single row (example: drop current and load the changed row)

  This behaviour can be configured easily into a dlt rersource:
  - Pass the incremental column as a parameter inside the resource
    ```python
    def rest_api_notion_incremental(
      last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
    ):
    ```
    We choose the column `last_edited_time` since it keeps track of whenever a paragraph changes.
  - Pass the following arguments inside `@dlt.resource` to define the strategy for dealing with duplicate rows:
    - `write_disposition="merge"`: ensures that any duplicate rows are merged on the primary key
    - `primary_key="block_id"`: specifies the primary key that we'd like to merge on. In our case, this is `block_id`, which is a unique id corresponding to each block (paragraph).
    - `columns={"last_edited_time":{"dedup_sort":"desc"}}`: this specifies the deduplication strategy (how we would like to resolve duplicate rows). Here we chose to keep the row with the largest value of `last_edited_time`.

    Putting it together:

    ```python
    @dlt.resource(
        name="employee_handbook",
        write_disposition="merge",
        primary_key="block_id",
        columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
    def rest_api_notion_incremental(
        last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
    ):
        for block in rest_api_notion_resource.add_map(extract_page_content):   
            if not(len(block["content"])):
                continue
            yield block
  ```
  Here, `rest_api_notion_resoure` yields the JSON response from the Notion REST API and `extract_page_content` is the transformation function that we pass the JSON response through.

5. Create the pipeline and run it

  With our source configured, we can now define the pipeline and run it.

  Normally, to do this we would run  
  ```python
  pipeline.run(
    rest_api_notion_incremental,
    table_name="employee_handbook",
    write_disposition="merge"
  )
  ```
  and this would load the data into lancedb normally, without creating any embeddings.

  However, we can have lancedb automatically create embeddings and load it along with the normal data using dlt's native adapter for lancedb: `lancedb_adapter`. It will use the embedding model that we specified in the credentials.   
    
  ```python
  pipeline.run(
    lancedb_adapter(
      rest_api_notion_incremental,
      embed="content" # The column that we'd like to embed
    )
    table_name="employee_handbook",
    write_disposition="merge"
  )
  ```

Run this block:

In [25]:
dlt.secrets["sources.rest_api.notion.api_key"]

'secret_q4QgENGc9JKXgGContzdV6Z6KMTwzLYXWX9VMFi1dlb\r\n'

In [24]:
f"Bearer {dlt.secrets['sources.rest_api.notion.api_key']}"

'Bearer secret_q4QgENGc9JKXgGContzdV6Z6KMTwzLYXWX9VMFi1dlb\r\n'

### 5. Run the pipeline

In [50]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="homework_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": "secret_q4QgENGc9JKXgGContzdV6Z6KMTwzLYXWX9VMFi1dlb"
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="homework_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework_handbook",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

Pipeline company_policies load step completed in ---
0 load package(s) were loaded to destination LanceDB and into dataset None
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7ff8ea9039d0> location to store data


### 6. Visualize the output

In [51]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,1c9b58c7-08bd-5f51-82f2-28ed84a760f5,"[-0.024265556, 0.04746074, -0.01179647, 0.0638...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-22 07:26:25.018252+00:00,1721633181.7005467,6NA5Cto3uvqZ5w
1,11112dc9-45b7-5124-b24d-bae0fcec7492,"[-0.04966156, 0.10853508, -0.009762607, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-22 07:26:25.022300+00:00,1721633181.7005467,wrBjGWByhv6H1A
2,f3ea175a-fa2c-56d7-ad78-2875f22c0108,"[-0.06316319, 0.17331503, 0.025351755, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.022643+00:00,1721633181.7005467,jiMET/1Ceq/Ycg
3,cee4ab9e-37fc-5e94-83fc-d0ccc70f1205,"[-0.109743185, 0.10586075, 0.003290699, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.022906+00:00,1721633181.7005467,LZRvlRR9L5Xg6w
4,671b2ac3-a3e4-5380-a939-2604c62fc95f,"[0.05242333, -0.064576, 0.06586297, 0.01454380...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.023175+00:00,1721633181.7005467,9FmffTX2vmKhzQ
5,2627eee4-2211-5162-a4a7-a286097bfd52,"[0.00052337867, -0.054883413, 0.043573413, -0....",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.023480+00:00,1721633181.7005467,ShOBhhtx3Y+OCA
6,b3e51b1c-c39a-50eb-8572-b4ec91d17db8,"[0.03802633, -0.021509705, 0.04752782, 0.06470...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.023759+00:00,1721633181.7005467,cHHt0ASaSO/NHw
7,27476ec7-0b56-56e1-952e-c10c4903d873,"[-0.05858811, -0.07540446, 0.033775203, 0.0096...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.024042+00:00,1721633181.7005467,L4BHgbZUIWmHRA
8,23fffa8e-a8b7-58fb-bc1a-1b303f6a5607,"[-0.013599302, 0.0047530197, 0.024835136, 0.01...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-22 07:26:25.024362+00:00,1721633181.7005467,Tedy47C2SsK62A
9,fa2989b7-d59a-5cc0-824c-1d980b964d5d,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.024625+00:00,1721633181.7005467,h2f0HYAmJAyccQ


 ---

Now we make change to one of the paragraphs and run the pipeline again to see the effect of incremental loading. We observe two things:
1. The column `inserted_at_time` only changed for the updated row, implying that only this row was added
2. Looking at the primary key `block_id` we see that the original row was dropped and the updated row was inserted

In [52]:
db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___homework_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,1c9b58c7-08bd-5f51-82f2-28ed84a760f5,"[-0.024265556, 0.04746074, -0.01179647, 0.0638...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-22 07:26:25.018252+00:00,1721633181.7005467,6NA5Cto3uvqZ5w
1,11112dc9-45b7-5124-b24d-bae0fcec7492,"[-0.04966156, 0.10853508, -0.009762607, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-22 07:26:25.022300+00:00,1721633181.7005467,wrBjGWByhv6H1A
2,f3ea175a-fa2c-56d7-ad78-2875f22c0108,"[-0.06316319, 0.17331503, 0.025351755, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.022643+00:00,1721633181.7005467,jiMET/1Ceq/Ycg
3,cee4ab9e-37fc-5e94-83fc-d0ccc70f1205,"[-0.109743185, 0.10586075, 0.003290699, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.022906+00:00,1721633181.7005467,LZRvlRR9L5Xg6w
4,671b2ac3-a3e4-5380-a939-2604c62fc95f,"[0.05242333, -0.064576, 0.06586297, 0.01454380...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.023175+00:00,1721633181.7005467,9FmffTX2vmKhzQ
5,2627eee4-2211-5162-a4a7-a286097bfd52,"[0.00052337867, -0.054883413, 0.043573413, -0....",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.023480+00:00,1721633181.7005467,ShOBhhtx3Y+OCA
6,b3e51b1c-c39a-50eb-8572-b4ec91d17db8,"[0.03802633, -0.021509705, 0.04752782, 0.06470...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.023759+00:00,1721633181.7005467,cHHt0ASaSO/NHw
7,27476ec7-0b56-56e1-952e-c10c4903d873,"[-0.05858811, -0.07540446, 0.033775203, 0.0096...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.024042+00:00,1721633181.7005467,L4BHgbZUIWmHRA
8,23fffa8e-a8b7-58fb-bc1a-1b303f6a5607,"[-0.013599302, 0.0047530197, 0.024835136, 0.01...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-22 07:26:25.024362+00:00,1721633181.7005467,Tedy47C2SsK62A
9,fa2989b7-d59a-5cc0-824c-1d980b964d5d,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-22 07:26:25.024625+00:00,1721633181.7005467,h2f0HYAmJAyccQ


## Part 2: Create a RAG bot using Ollama

With the contents from the employee handbook vectorized and stored in LanceDB, we're now ready to create our RAG with Ollama.



What is RAG?

Retrieval Automated Generation (RAG) is the framework of retrieving relevant documents from a database and passing it along with a query into an LLM so that the LLM can generate context-aware responses.

In our case, if we were to ask an LLM questions about our specific employee policies, then we would not get useful responses because the LLM has never seen these policies. A solution to this could be to paste all of the policies into the prompt and then ask our questions. However, this would not be feasible given the limitations on the size of the context window.

We can bypass this limitation using RAG:
1. Given a user question, we would first embed this question into a vector
2. Then we would do a vector search on our LanceDB table and retrieve top k results - which would be the most relevant paragraphs corresponding to the question
3. Finally, we would pass the original question along with the retrieved paragraphs as a prompt into the LLM


1. Install Ollama into the notebook's local runtime

In [53]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


2. Start Ollama using `ollama serve`. This needs to run in the backgound - so we run it using `nohup` (to see the output log, open nohup.out).

In [54]:
!nohup ollama serve > nohup.out 2>&1 &

3. Pull the desired model. We're going to be using `llama1-uncensored` (takes about 1m to download)

In [55]:
%%capture
!ollama pull llama2-uncensored

In this next part we're going to be writing functions that accept user question, retrieve the relevant paragraphs from lancedb, and the pass the question and the retrieved pages as input into the ollama chat assistant

4. pip install ollama and import it

In [56]:
!pip install ollama

In [57]:
import ollama

5. Write a function that can retrieve content from lancedb relevant to the user query
  
  With LanceDB, you don't have to explicity embed the question. LanceDB stores information on the embedding model used and automatically embeds the question.

  We use the `db_table.search()` function to query the DB and then limit it to the top 2 most similar results and return that as the context to pass to the RAG.

  Limiting results is important because otherwise there might be too much confusing information. Similarly only picking the top choice might not give enough information.

In [58]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

6. Finally we define a very basic RAG. We define a simple system prompt, retrieve the relevant context for the user query with the function defined above and then send the user question and the context to the `llama2-uncensored` model.

In [59]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework_handbook")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [44]:
len(dbtable)

17

In [47]:
dbtable.to_pandas().agg({"last_edited_time": "max"})

last_edited_time   2024-07-05 23:33:00+00:00
dtype: datetime64[ns, UTC]

And we run the RAG! Some example questions you can ask:

* How many vacation days do I get?
* Can I get maternity leave?

**Note**: This is a very basic implementation of a RAG, since this workshop is mainly about data ingestion. So expect some weird answers. If you do stop and restart the cell, you will need to rerun the cell containing `ollama serve` first.

In [ ]:
main()

You:  how many PTO days are the employees entitled to in a year?
Assistant: Based on the provided context, it seems like the employee is asking for more clarification about PTO days and how they accrue. They can contact HR for information missing in the paragraph or any details that need further explanation.
You:  how many PTO days?
Assistant: Based on the provided context, it seems like the employee is asking for clarification about PTO days and how they accrue. The employee can contact HR if there is any further information that needs explanation or clarification.


There's a lot more to learn and do with dlt and LanceDB, find more info the [dlt docs](https://dlthub.com/docs/) and the [LanceDB docs](https://lancedb.github.io/lancedb/)

If you have questions about this workshop or dlt, feel free to join our [community on Slack](https://dlthub.com/community).

If you're at EuroPython in Prague this week, come see us at our booth!